In [1]:
import pandas as pd
from geopy.distance import geodesic
import math
import getweatherdata
import os


def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    delta_lon = lon2 - lon1

    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
                                           * math.cos(lat2) * math.cos(delta_lon))

    initial_bearing = math.atan2(x, y)
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = int((initial_bearing + 360) % 360)

    return compass_bearing


def calculate_distance(row):
    return geodesic((row["previous_latitude"], row["previous_longitude"]), (row["latitude"], row["longitude"])).meters


def get_weather_data(row):
    lat = float(row["latitude"])
    lon = float(row["longitude"])
    timestamp = row["date"]
    filename = row["filename"]
    result = getweatherdata.main(filename, lat, lon, timestamp)
    return result


df = pd.read_csv(os.path.join("data", "flight_data.csv"))
df.head()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3478 entries, 0 to 3477
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   filename             3478 non-null   object 
 1   date                 3478 non-null   object 
 2   pilot                3089 non-null   object 
 3   time                 3478 non-null   object 
 4   latitude             3478 non-null   float64
 5   longitude            3478 non-null   float64
 6   gps_altitude_m       3478 non-null   int64  
 7   pressure_altitude_m  3478 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 217.5+ KB


,filename,date,pilot,time,latitude,longitude,gps_altitude_m,pressure_altitude_m
0,2013-10-26-XGD-001-01.igc,2013-10-26,NaN,08:47:00,36.539000,29.169517,1643,1703
1,2013-10-26-XGD-001-01.igc,2013-10-26,NaN,08:47:10,36.538667,29.168733,1635,1695
2,2013-10-26-XGD-001-01.igc,2013-10-26,NaN,08:47:20,36.538400,29.167917,1625,1684
3,2013-10-26-XGD-001-01.igc,2013-10-26,NaN,08:47:30,36.538267,29.166950,1617,1676
4,2013-10-26-XGD-001-01.igc,2013-10-26,NaN,08:47:40,36.538683,29.166100,1618,1676


In [2]:
df["date"] = pd.to_datetime(df["date"] + " " + df["time"])
df["elapsed_time"] = (df["date"] - df["date"].iloc[0]).dt.total_seconds()
df["previous_latitude"] = df["latitude"].shift(1)
df["previous_longitude"] = df["longitude"].shift(1)
df.drop(df.index[0], inplace=True)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / df["date"].diff().dt.total_seconds()
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["speed_km/s"] = ((df["distance_m"]/1000) /
                    (df["date"].diff().dt.total_seconds()/3600))
df["bearing"] = df.apply(lambda row: calculate_bearing(
    row["previous_latitude"], row["previous_longitude"],
    row["latitude"], row["longitude"]
), axis=1)
df["glide_ratio"] = df.apply(
    lambda row: row["distance_m"] /
    abs(row["climb_m"]) if row["climb_m"] != 0 else 0,
    axis=1
)


In [3]:
df[["temp", "pressure", "humidity", "dew_point", "wind_speed", "wind_deg"]] = df.apply(get_weather_data, axis=1, result_type="expand")
df.fillna(0, inplace=True)
df.drop(["filename", "pilot", "time", "previous_latitude", "previous_longitude",
        "climb_m", "distance_m"], axis=1, inplace=True)
df.to_csv(os.path.join("data", "flight_data_processed.csv"), index=False)

df.head()

,date,latitude,longitude,gps_altitude_m,pressure_altitude_m,elapsed_time,climb_rate_m/s,speed_km/s,bearing,glide_ratio,temp,pressure,humidity,dew_point,wind_speed,wind_deg
1,2013-10-26 08:47:10,36.538667,29.168733,1635,1695,10.0,0.0,0.000000,242,0.000000,13.48,1018.0,31.0,-2.92,1.0,290.0
2,2013-10-26 08:47:20,36.538400,29.167917,1625,1684,20.0,-1.0,28.400413,247,7.889004,13.48,1018.0,31.0,-2.92,1.0,290.0
3,2013-10-26 08:47:30,36.538267,29.166950,1617,1676,30.0,-0.8,31.614276,260,10.977179,13.48,1018.0,31.0,-2.92,1.0,290.0
4,2013-10-26 08:47:40,36.538683,29.166100,1618,1676,40.0,0.1,32.060870,301,89.057972,13.48,1018.0,31.0,-2.92,1.0,290.0
5,2013-10-26 08:47:50,36.539283,29.165633,1627,1686,50.0,0.9,28.299159,327,8.734308,13.48,1018.0,31.0,-2.92,1.0,290.0
